the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

changelog:

* 4/16/19: added newagg
* 4/17/19: reformatted the ordering of how code runs, and variable names. added aggregation #3.
* 4/19/19: changed the standardizing so that log(x+1) is now applied prior to standardization. also removed ordinal variables from standardizing algorithm and concat them in later with median 0 and iqr 1 so standardize value is either 0 or 1. values from ordinal are not log transformed.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [40]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/04042019_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals, 14478 icustay_id
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 118 ms


In [41]:
#pd.to_datetime(final_pt_df2['t_0'])- pd.to_datetime(final_pt_df2['t_0'])
#final_pt_df2
(pd.to_datetime(final_pt_df2['t_0'])-pd.to_datetime(final_pt_df2['t_0_sc'])).describe() #ssc time - ab time
#(pd.to_datetime(final_pt_df2['ICU_admit'])-pd.to_datetime(final_pt_df2['t_0'])).describe()

count                     14478
mean     0 days 01:55:28.064649
std      0 days 10:00:09.874180
min           -1 days +00:00:00
25%           -1 days +19:37:00
50%             0 days 02:45:00
75%             0 days 08:39:00
max             1 days 00:00:00
dtype: object

time: 17.6 ms


In [42]:
#final_pt_df2
(pd.to_datetime(final_pt_df2['t_0'])-pd.to_datetime(final_pt_df2['ICU_admit'])).describe() #icu admit time - ab time

count                     14478
mean     0 days 06:52:07.033775
std      0 days 17:39:21.503063
min           -1 days +12:00:21
25%           -1 days +21:47:17
50%      0 days 02:56:52.500000
75%      0 days 08:05:23.250000
max             3 days 23:46:07
dtype: object

time: 14.9 ms


In [43]:
(pd.to_datetime(final_pt_df2['t_0_sc'])-pd.to_datetime(final_pt_df2['ICU_admit'])).describe() #icu admit time - ssc time

count                       14478
mean       0 days 04:56:38.969125
std        0 days 20:01:44.531744
min             -2 days +12:08:23
25%      -1 days +20:15:15.500000
50%             -1 days +23:05:58
75%        0 days 05:21:32.250000
max               4 days 21:07:19
dtype: object

time: 16.5 ms


In [44]:
#importing in all clinical_variable files
date= '04042019'
os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
big_df= pd.read_csv(Path(wd+'/data/processed/merged/%s_longdf_preImp.csv' %(date),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]

time: 11.6 s


In [45]:
#removing firstpos else neg ssc col
big_df=big_df.loc[:,list(big_df.columns!="first_pos_else_neg_ssc")]
#changing pao2/fio2 ratio label
big_df.loc[big_df['label']=="pao2/fio2",'label']="pao2fio2Ratio"

time: 1.42 s


In [46]:
big_df.groupby('label')['value'].describe().sort_values('unique')
##categorical variables:
#rrt, phenylephrine, norepinephrine, vasopressin, dobutamine, dopamine, epinephrine, leukocyte, vent_recieved


,count,unique,top,freq
label,,,,
leukocyte,14478,2,Neg/Not_tested,12278
norepinephrine,34427,2,1.0,22788
o2_flow,14734,2,0.0,13734
gender,14478,2,M,7940
phenylephrine,26635,2,1.0,14378
epinephrine,16080,2,0.0,14188
dopamine,19236,2,0.0,13623
nitrite,14478,2,Neg/Not_tested,14061
dobutamine,15767,2,0.0,14247


time: 1.74 s


In [47]:
big_df[big_df['label']=='vent_recieved']['value'].value_counts() #
#df["B"] = df["A"].astype('category')

#was origionally:
# None      8387
# Oxygen    3986
# Mech      2295

# now is:
# Mech      7144
# Oxygen    6176
# None       748

Mech      7144
Oxygen    6176
None       748
Name: value, dtype: int64

time: 443 ms


In [48]:
big_df[big_df['label']=='vent_recieved']['icustay_id'].nunique()
big_df.head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id
0,200012.0,1,daily_sofa_score,daily_sofa,0 days 00:00:00.000000000,2153-12-23,sofa,28448
1,200012.0,0.0,y/n,dobutamine,0 days 00:00:00.000000000,2153-12-23,dobutamine,28448
2,200012.0,0.0,y/n,dopamine,0 days 00:00:00.000000000,2153-12-23,dopamine,28448
3,200012.0,0.0,y/n,epinephrine,0 days 00:00:00.000000000,2153-12-23,epinephrine,28448
4,200012.0,asian,NaN,ethnicity,0 days 00:00:00.000000000,2153-12-23,pt_info,28448


time: 476 ms


In [49]:
big_df[big_df['label']=='vent_recieved'].head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id
13,200012.0,Oxygen,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2153-12-23,ventcategory,28448
109,200014.0,Mech,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2105-02-17,ventcategory,9514
500,200019.0,Mech,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2178-07-09,ventcategory,21789
850,200030.0,Oxygen,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2150-11-14,ventcategory,14987
1482,200033.0,Mech,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2198-08-11,ventcategory,56369


time: 442 ms


In [50]:
# #converting venttype to category
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')
# #big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 824 µs


In [51]:
#converting categories to integers
vent_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])[1]
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']=pd.factorize(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])[0]#= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')

gender_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='gender','value'])[1]
big_df.loc[big_df.loc[:,'label']=='gender','value']=pd.factorize(big_df.loc[big_df.loc[:,'label']=='gender','value'])[0] #male=1

leukocyte_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[1]
big_df.loc[big_df.loc[:,'label']=='leukocyte','value']=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[0] 

nitrite_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[1]
big_df.loc[big_df.loc[:,'label']=='nitrite','value']=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[0] 


time: 6.71 s


# changing pao2/fio2 ratio to a category
A PaO2/FiO2 ratio less than or equal to 200 is necessary for the diagnosis of acute respiratory distress syndrome by the AECC criteria.[6] 
The more recent Berlin criteria defines mild ARDS at a ratio of <300.

A PaO2/FiO2 ratio less than or equal to 250 is one of the minor criteria for severe community acquired pneumonia (i.e., possible indication for inpatient treatment).

A PaO2/FiO2 ratio less than or equal to 333 is one of the variables in the SMART-COP risk score for intensive respiratory or vasopressor support in community-acquired pneumonia.



In [52]:
pd.to_numeric(big_df[big_df['label']=="pao2fio2Ratio"]['value'])
labels= ['0-200', '201-333', "334-475","476+"]
bins = pd.IntervalIndex.from_tuples([(0, 200), (200, 333), (333, 475),(475,3000)])

big_df.loc[big_df['label']=="pao2fio2Ratio",'value']=pd.cut(pd.to_numeric(big_df.loc[big_df['label']=="pao2fio2Ratio",'value']), bins,right=False, labels=labels)
big_df.loc[big_df['label']=="pao2fio2Ratio",'value'].value_counts()


(475, 3000]    9911
(0, 200]       1885
(200, 333]     1422
(333, 475]     1260
Name: value, dtype: int64

time: 2.14 s


## calc median/iqr for standardization

In [53]:
continuous=['daily_sofa',
            'lactate',
            'mingcs',
            'diasbp',
            'heartrate',
            'meanartpress',
            'resprate',
            'sysbp',
            'temperature',
            'hemoglobin',
            'platelet',
            'wbc',
            'calcium',
            'glucose',
            'ph',
            'bicarbonate',
            'bun',
            'chloride',
            'creatinine',
            'inr',
            'potassium',
            'ptt',
            'sodium',
            'bilirubin',
            'spo2']

sparse=['bands','pco2' ]

onetime=['first_admit_age','height','weight']

vaso_active=['phenylephrine',
            'norepinephrine',
            'vasopressin',
            'dobutamine',
            'dopamine',
            'epinephrine'] 

ordinal=[
            'leukocyte',
            'nitrite',
            'vent_recieved',
            'o2_flow',
            'rrt',
            'pao2fio2Ratio',
            #"all_vasoactives"
]


time: 10.7 ms


In [54]:
final_bin=final_pt_df2[['icustay_id',"final_bin"]].copy()
#list of healthy pt
healthy_pt=list(final_bin[final_bin['final_bin']=="C_neg/A_partial"]['icustay_id'])
del(final_bin)

time: 5.96 ms


In [55]:
#formatting big_df to be limited to only numeric columns
#norm_df=big_df[big_df['label'].isin(label_list)].copy() #limiting to numerical variables
norm_df=big_df.copy()

categorical_dropped= ['ethnicity'] #dropping ethnicity from model
norm_df=norm_df[norm_df['label'].isin(continuous+onetime+sparse)]

norm_df['value']= pd.to_numeric(norm_df['value']) #converting to numeric 

#healthy patients filter
healthy_df=norm_df[norm_df['icustay_id'].isin(healthy_pt)].copy() #only numerical values for cneg/ab partial pt

time: 6.16 s


In [56]:
#finding the last 24 hours of each healthy patient.
healthy_pt_end=pd.DataFrame(healthy_df.groupby("icustay_id")['delta'].max())

healthy_pt_end["start"]=healthy_pt_end['delta']- pd.to_timedelta("1 day 00:00:00")
healthy_pt_end=healthy_pt_end.rename(columns={"delta":"end"}).reset_index()

healthy_df= pd.merge(healthy_df,healthy_pt_end, left_on="icustay_id", right_on="icustay_id", how="left" ) #now have the last 24 hours annotated for each patient as start and end.
healthy_df.head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id,end,start
0,200012.0,1.00,daily_sofa_score,daily_sofa,0 days 00:00:00.000000000,2153-12-23,sofa,28448,0 days 15:00:00.000000000,-1 days +15:00:00
1,200012.0,32.99,years,first_admit_age,0 days 00:00:00.000000000,2153-12-23,pt_info,28448,0 days 15:00:00.000000000,-1 days +15:00:00
2,200012.0,51.20,kg,weight,0 days 00:00:00.000000000,2153-12-23,weight,28448,0 days 15:00:00.000000000,-1 days +15:00:00
3,200012.0,1.30,mmol/L,lactate,0 days 02:36:00.000000000,2153-12-23,labs,28448,0 days 15:00:00.000000000,-1 days +15:00:00
4,200012.0,15.00,gcs_score,mingcs,0 days 05:12:00.000000000,2153-12-23,gcs,28448,0 days 15:00:00.000000000,-1 days +15:00:00


time: 1.77 s


In [57]:
##qc, trying to log before standardization
pd.DataFrame(np.log(healthy_df.groupby("label")['value'].median())).reset_index()

,label,value
0,bands,1.386294
1,bicarbonate,3.218876
2,bilirubin,-0.223144
3,bun,2.995732
4,calcium,0.131028
5,chloride,4.653960
6,creatinine,0.000000
7,daily_sofa,1.098612
8,diasbp,4.077537
9,first_admit_age,4.177459


time: 847 ms


In [58]:
#calculating medians and iqr for each label based on healthy patient's last 24 hours in icu
median_label=pd.DataFrame((healthy_df.groupby("label")['value'].median())).reset_index()
median_label=median_label.rename(columns={'value':"median"})
iqr_label=pd.DataFrame((healthy_df.groupby("label")['value'].quantile(0.75)-healthy_df.groupby("label")['value'].quantile(0.25))).reset_index()
iqr_label=iqr_label.rename(columns={'value':"iqr"})
median_label=pd.merge(median_label,iqr_label)  #final median df
del(iqr_label)
        
#saving the median label for standardization, will be used later on in teh notebook
# pd.DataFrame(median_label).to_csv(Path(
#     wd+'/data/processed/merged/%s_healthy_medians.csv' %(date)),index=False)

time: 1.08 s


In [149]:
median_label

,label,median,iqr
0,bands,4.00,8.000000
1,bicarbonate,25.00,5.000000
2,bilirubin,0.80,1.600000
3,bun,20.00,21.000000
4,calcium,1.14,0.110000
5,chloride,105.00,8.000000
6,creatinine,1.00,0.900000
7,daily_sofa,3.00,3.000000
8,diasbp,59.00,18.000000
9,first_admit_age,65.20,26.065000


time: 12.8 ms


In [60]:
# #making a test df of one patient so i can mess around with it
# test_df=norm_df[norm_df['icustay_id']==200095.0].copy() #299995 

# #merging in medians/iqr
# test_df=pd.merge(test_df, median_label)
# test_df.head()
# test_df['standardize']=((test_df['value']-test_df['median'])/test_df['iqr']).fillna(0)
# #normalizing

time: 1.6 ms


In [144]:
continuous=['daily_sofa',
            'lactate',
            'mingcs',
            'diasbp',
            'heartrate',
            'meanartpress',
            'resprate',
            'sysbp',
            'temperature',
            'hemoglobin',
            'platelet',
            'wbc',
            'calcium',
            'glucose',
            'ph',
            'bicarbonate',
            'bun',
            'chloride',
            'creatinine',
            'inr',
            'potassium',
            'ptt',
            'sodium',
            'bilirubin',
            'spo2']

sparse=['bands','pco2' ]
onetime=['first_admit_age','height','weight']
vaso_active=['phenylephrine',
            'norepinephrine',
            'vasopressin',
            'dobutamine',
            'dopamine',
            'epinephrine']  #used to be in ordinal, seperated out so i could combine them into all_vasoactives

ordinal=[
            'leukocyte',
            'nitrite',
            'vent_recieved',
            'o2_flow',
            'rrt', 
            "all_vasoactives",
            'pao2fio2Ratio'
]

categorical=[
            "ethnicity",
            'gender'
]


time: 34.2 ms


# Standardizing data

### dataformatting: convert all dtypes to a numeric type that pereserves nan. 

In [145]:
#splitting categorical, ordinal and continuous
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy() 

#continuous and ordinal variables
big_noCat1= big_df.loc[big_df.loc[:,'label'].isin(continuous+sparse),:].copy() # do i need to add sparse?
big_noCat1['value']= big_noCat1['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 

time: 2min 34s


### adding a standardized value (x-median)/iqr  where median is of the last 24 hours in time window for culture neg/ ab partial patients 

In [146]:
big_noCat=pd.merge(big_noCat1, median_label, how="left") #

#loging values
big_noCat['median']= np.log(big_noCat['median']+1.0)
big_noCat['iqr']= np.log(big_noCat['iqr']+1.0)
big_noCat['raw_value']=big_noCat['value']
big_noCat['value']=np.log(big_noCat['value']+1.0)
##

time: 1.92 s


In [147]:
big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0) #standardize is log standardized

#making an ordinal df to concat on
ord_df=big_df.loc[big_df.loc[:,'label'].isin(ordinal+vaso_active),:].copy()
ord_df['raw_value']=ord_df['value']
ord_df['standardize']=ord_df['value']
ord_df['median']=None
ord_df['iqr']=None

big_noCat=pd.concat([big_noCat,ord_df], sort=False)
del(ord_df)

#making all ordinal values in standardize equal to unstandardized
big_noCat.loc[big_noCat['label'].isin(ordinal+vaso_active),'standardize']=big_noCat.loc[big_noCat['label'].isin(ordinal+vaso_active),'value']  #do i need to add sparse?
big_noCat.head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id,median,iqr,raw_value,standardize
0,200012.0,0.693147,daily_sofa_score,daily_sofa,0 days 00:00:00.000000000,2153-12-23,sofa,28448,1.38629,1.38629,1,-0.5
1,200012.0,0.832909,mmol/L,lactate,0 days 02:36:00.000000000,2153-12-23,labs,28448,0.993252,0.875469,1.3,-0.183151
2,200012.0,2.77259,gcs_score,mingcs,0 days 05:12:00.000000000,2153-12-23,gcs,28448,2.70805,1.09861,15,0.0587455
3,200012.0,4.38203,mmHg,diasbp,0 days 06:16:00.000000000,2153-12-23,vitals,28448,4.09434,2.94444,79,0.0977035
4,200012.0,4.70048,bpm,heartrate,0 days 06:16:00.000000000,2153-12-23,vitals,28448,4.45435,3.17805,109,0.0774477


time: 8.35 s


In [148]:
big_noCat['standardize']= big_noCat['standardize'].apply(pd.to_numeric, args=('coerce',)) #errors occuring downstream due to not having numeric, trying this 4/18/19

time: 28.9 s


### qc below, delete ###
exploring if loging the data is worth doign before standardizing due to many of the variable distributions being left skewed without it. 

In [51]:
#qc, seeing if log() is worth doing. 
big_noCat=pd.merge(big_noCat, median_label, how="left")
big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0)

time: 3.73 s


In [52]:
((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).describe().round(2)

count    6726715.00
mean           0.11
std            0.97
min          -24.00
25%           -0.46
50%            0.00
75%            0.57
max           74.50
dtype: float64

time: 578 ms


In [53]:
((np.log(big_noCat['value']+1)-np.log(big_noCat['median']+1))/np.log(big_noCat['iqr']+1)).fillna(0).describe().round(2)

count    6943374.00
mean           0.01
std            0.15
min           -5.15
25%           -0.04
50%            0.00
75%            0.04
max            8.69
dtype: float64

time: 561 ms


In [54]:
big_noCat['standardize'].max()#.round(2)

74.49999999999999

time: 46 ms


In [55]:
big_noCat.head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id,median,iqr,standardize
0,200012.0,1.0,daily_sofa_score,daily_sofa,0 days 00:00:00.000000000,2153-12-23,sofa,28448,3.0,3.0,-0.666667
1,200012.0,0.0,y/n,dobutamine,0 days 00:00:00.000000000,2153-12-23,dobutamine,28448,NaN,NaN,0.000000
2,200012.0,0.0,y/n,dopamine,0 days 00:00:00.000000000,2153-12-23,dopamine,28448,NaN,NaN,0.000000
3,200012.0,0.0,y/n,epinephrine,0 days 00:00:00.000000000,2153-12-23,epinephrine,28448,NaN,NaN,0.000000
4,200012.0,0.0,y/n,norepinephrine,0 days 00:00:00.000000000,2153-12-23,norepinephrine,28448,NaN,NaN,0.000000


time: 20 ms


In [56]:
##wbc has a very high variance, need to explore:
big_noCat[big_noCat['standardize']==74.49999999999999] #so wbc has high variance.

big_noCat[big_noCat['label']=='wbc']['value'].describe() #4,500 to 11,000 WBC per microliter (4.5 to 11.0 × 109/L).
#our highest value is 471.700000
#The normal range for WBC is 5 to 10 K/uL. 

#what are the units= K/uL
#big_noCat[big_noCat['label']=='wbc']


big_noCat[big_noCat['label']=='wbc'][['value','source']]#.sort_values(by='value',ascending=False)

big_noCat[big_noCat['label']=='wbc']['source'].describe() #all wbc are from labs

count     55399
unique        1
top        labs
freq      55399
Name: source, dtype: object

time: 1.94 s


In [57]:
high_wbc=list(set((big_noCat[(big_noCat['label']=='wbc')& (big_noCat['value']>100)]['icustay_id']))) #24 patients with wbc >100
final_pt_df2[final_pt_df2['icustay_id'].isin(high_wbc)]['final_bin'].value_counts()


C_neg/A_full       19
C_pos/A_full        3
C_neg/A_partial     2
Name: final_bin, dtype: int64

time: 560 ms


In [58]:
#visualizing
big_noCat['standardize'].transform(lambda x:np.log(x+1))
#big_noCat#.groupby('label')['standardize'].transform(lambda x:np.log(x+1))#.describe()

0         -1.098612
1          0.000000
2          0.000000
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11        -0.336472
12         0.405465
13         0.747214
14         0.714653
15         0.336472
16        -0.847298
17        -0.287682
18        -0.628609
19         0.453918
20         0.328504
21         0.448025
22        -0.223144
23         0.251314
24        -0.287682
25        -1.455287
26         0.265703
27         0.000000
28        -0.693147
29        -0.010694
             ...   
6943344    0.451985
6943345   -1.189584
6943346    0.133531
6943347   -1.749200
6943348   -0.559616
6943349    0.823200
6943350   -0.650588
6943351    0.615186
6943352    0.000000
6943353    0.641854
6943354   -0.832909
6943355    0.133531
6943356   -0.139762
6943357    0.133531
6943358    0.287682
6943359   -0.737599
6943360    0.223144
6943361    0.133531
6943362    0.287682


time: 9.08 s


In [59]:
##need to explore:
#  #icustay id:248526
# ptp_creatinine: 1.666667
# mean_creatinine: 15.40741
# std_creatinine:0.9049475
big_noCat[(big_noCat['icustay_id']==248526) &(big_noCat['label']=="creatinine")]
 
    
    
 #  #icustay id:233363
# ptp_creatinine: 13.11111
# mean_creatinine: 9.574074
# std_creatinine:5.281889
big_noCat[(big_noCat['icustay_id']==248526) &(big_noCat['label']=="creatinine")]

,icustay_id,value,uom,label,delta,t_0,source,subject_id,median,iqr,standardize
3339676,248526.0,14.5,mg/dL,creatinine,0 days 10:28:00.000000000,2157-03-01,labs,75557,1.0,0.9,15.000000
3339781,248526.0,14.3,mg/dL,creatinine,1 days 01:20:00.000000000,2157-03-01,labs,75557,1.0,0.9,14.777778
3339949,248526.0,15.8,mg/dL,creatinine,2 days 04:04:00.000000000,2157-03-01,labs,75557,1.0,0.9,16.444444


time: 998 ms


## qc above, delete##

# aggregation1:
### clincally guided min/max

In [60]:
hi_value= ['bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine',
           'first_admit_age','leukocyte','nitrite','vent_recieved','o2_flow' ]

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp',
'pao2fio2Ratio']

both_value=['calcium',
'sodium',
'wbc']


time: 9.58 ms


In [61]:
#lambda x: x if not np.isnan(x) else -1

dict1={str(x):max for x in hi_value}
dict2={str(x):min for x in low_value } #dict2={str(x):min for x in low_value }
dict3={str(x):[max,min] for x in both_value }

time: 3.3 ms


In [62]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

dict1 = merge_two_dicts(dict1, dict2)
dict1 = merge_two_dicts(dict1, dict3)

time: 5.88 ms


In [63]:
dict1

{'bands': <function max>,
 'bicarbonate': <function min>,
 'bilirubin': <function max>,
 'bun': <function max>,
 'calcium': [<function max>, <function min>],
 'chloride': <function max>,
 'creatinine': <function max>,
 'daily_sofa': <function max>,
 'diasbp': <function min>,
 'dobutamine': <function max>,
 'dopamine': <function max>,
 'epinephrine': <function max>,
 'first_admit_age': <function max>,
 'glucose': <function max>,
 'heartrate': <function max>,
 'hemoglobin': <function min>,
 'inr': <function max>,
 'lactate': <function max>,
 'leukocyte': <function max>,
 'meanartpress': <function min>,
 'mingcs': <function min>,
 'nitrite': <function max>,
 'norepinephrine': <function max>,
 'o2_flow': <function max>,
 'pao2fio2Ratio': <function min>,
 'pco2': <function max>,
 'ph': <function min>,
 'phenylephrine': <function max>,
 'platelet': <function min>,
 'potassium': <function max>,
 'ptt': <function max>,
 'resprate': <function max>,
 'rrt': <function max>,
 'sodium': [<function 

time: 6.39 ms


# running all of the min/max aggregations.



In [64]:
##choose here if wanna use standardization or raw values.
values="standardize"

time: 1.05 ms


In [65]:
big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
table = pd.pivot_table(big_max, values=values, columns='label', index=['icustay_id'],aggfunc=max, dropna=False)

time: 1.28 s


In [66]:
big_max[big_max['label']=='leukocyte']['value'].describe() #good the categorical stuff worked

count    14478.000000
mean         0.151955
std          0.358989
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: value, dtype: float64

time: 252 ms


In [67]:
big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
table2 = pd.pivot_table(big_min, values=values, columns='label', index=['icustay_id'],aggfunc=min, dropna=False)

time: 1.5 s


In [68]:
big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
table3 = pd.pivot_table(big_both, values=values, columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)

time: 477 ms


In [69]:
print(len(table),len(table2),len(table3)) #14668 14668 14618

14478 14478 14417
time: 1.02 ms


In [70]:
#first left join all different aggregations together. 
worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')
worst_df.head()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,icustay_id,bands,bilirubin,bun,chloride,creatinine,daily_sofa,dobutamine,dopamine,epinephrine,...,ph,platelet,spo2,sysbp,"(max, calcium)","(max, sodium)","(max, wbc)","(min, calcium)","(min, sodium)","(min, wbc)"
0,200012.0,0.875,NaN,NaN,NaN,NaN,-0.666667,0.0,0.0,0.0,...,NaN,-0.450382,-0.50,-0.766667,NaN,NaN,-0.790323,NaN,NaN,-0.790323
1,200014.0,NaN,NaN,0.190476,1.125,-0.222222,-0.333333,0.0,0.0,0.0,...,-0.222222,-0.572519,-0.50,-1.600000,0.909091,0.8,0.725806,-0.090909,-0.2,0.145161
2,200019.0,NaN,NaN,0.190476,0.500,0.000000,1.000000,0.0,0.0,0.0,...,-0.111111,-0.396947,-8.75,-1.633333,NaN,0.2,0.596774,NaN,0.2,0.483871
3,200030.0,NaN,8.5625,0.380952,0.750,0.000000,0.666667,0.0,0.0,0.0,...,-0.111111,-0.229008,-1.25,-0.366667,1.727273,0.4,0.661290,0.090909,-0.2,0.435484
4,200033.0,NaN,NaN,-0.190476,0.375,-0.333333,-0.666667,0.0,0.0,0.0,...,0.888889,-0.595420,-0.25,-0.633333,-0.727273,1.2,-0.338710,-1.000000,-1.2,-0.580645


time: 70.7 ms


In [71]:
### formatting categorical to wide format to match the tables/worst_df
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy()
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per patient per time. 

time: 644 ms


In [72]:
def merge_cat_agg(num_df, cat_df, twoclass=False):
    """
    merges the categorical and aggregated dataframes together. if twoclass=True (default=False), returns just the truepos trueneg twoclass patients.
    """
    
    worst_df=pd.merge(num_df, cat_df, left_on='icustay_id', right_on='icustay_id',how='left')
#     worst_df['leukocyte']=worst_df['leukocyte'].astype("category")
#     worst_df['nitrite']=worst_df['nitrite'].astype("category")
#     worst_df['vent_recieved']=worst_df['vent_recieved'].astype("category")
    worst_df['ethnicity']=worst_df['ethnicity'].astype("category")
    worst_df['gender']=worst_df['gender'].astype("category")
    
    baselinePts= final_pt_df2.loc[final_pt_df2.loc[:,'final_bin'].isin(["C_pos/A_full","C_neg/A_partial"]),"icustay_id"]
    
    if twoclass==True:
        worst_df= worst_df.loc[worst_df.loc[:,'icustay_id'].isin(baselinePts),:]
    return(worst_df)


time: 16.8 ms


In [73]:
worst_df=merge_cat_agg(worst_df, big_categorical, twoclass=False) #using max/min aggregates 


time: 30.1 ms


In [74]:
list(worst_df)

['icustay_id',
 'bands',
 'bilirubin',
 'bun',
 'chloride',
 'creatinine',
 'daily_sofa',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'glucose',
 'heartrate',
 'inr',
 'lactate',
 'leukocyte',
 'nitrite',
 'norepinephrine',
 'o2_flow',
 'pco2',
 'phenylephrine',
 'potassium',
 'ptt',
 'resprate',
 'rrt',
 'temperature',
 'vasopressin',
 'vent_recieved',
 'bicarbonate',
 'diasbp',
 'hemoglobin',
 'meanartpress',
 'mingcs',
 'pao2fio2Ratio',
 'ph',
 'platelet',
 'spo2',
 'sysbp',
 ('max', 'calcium'),
 ('max', 'sodium'),
 ('max', 'wbc'),
 ('min', 'calcium'),
 ('min', 'sodium'),
 ('min', 'wbc'),
 'ethnicity',
 'gender']

time: 3.38 ms


In [75]:
worst_df

,icustay_id,bands,bilirubin,bun,chloride,creatinine,daily_sofa,dobutamine,dopamine,epinephrine,...,spo2,sysbp,"(max, calcium)","(max, sodium)","(max, wbc)","(min, calcium)","(min, sodium)","(min, wbc)",ethnicity,gender
0,200012.0,0.875,NaN,NaN,NaN,NaN,-0.666667,0.0,0.0,0.0,...,-0.50,-0.766667,NaN,NaN,-0.790323,NaN,NaN,-0.790323,asian,0
1,200014.0,NaN,NaN,0.190476,1.125,-0.222222,-0.333333,0.0,0.0,0.0,...,-0.50,-1.600000,0.909091,0.8,0.725806,-0.090909,-0.2,0.145161,unknown/other,1
2,200019.0,NaN,NaN,0.190476,0.500,0.000000,1.000000,0.0,0.0,0.0,...,-8.75,-1.633333,NaN,0.2,0.596774,NaN,0.2,0.483871,white/nonhispanic,0
3,200030.0,NaN,8.5625,0.380952,0.750,0.000000,0.666667,0.0,0.0,0.0,...,-1.25,-0.366667,1.727273,0.4,0.661290,0.090909,-0.2,0.435484,black,1
4,200033.0,NaN,NaN,-0.190476,0.375,-0.333333,-0.666667,0.0,0.0,0.0,...,-0.25,-0.633333,-0.727273,1.2,-0.338710,-1.000000,-1.2,-0.580645,white/nonhispanic,1
5,200036.0,NaN,NaN,0.523810,0.625,0.000000,-0.666667,0.0,0.0,0.0,...,-0.75,-0.733333,NaN,0.4,-0.500000,NaN,-0.6,-0.887097,white/nonhispanic,1
6,200055.0,NaN,-0.3125,0.952381,-0.625,8.777778,1.666667,0.0,0.0,0.0,...,-1.00,-2.166667,-0.272727,0.2,-0.564516,-0.272727,-0.4,-0.661290,black,0
7,200059.0,NaN,-0.0625,1.428571,0.500,0.555556,1.000000,0.0,0.0,0.0,...,-1.00,-1.666667,0.272727,0.2,0.806452,-1.181818,-1.4,-0.209677,white/nonhispanic,0
8,200061.0,NaN,3.1875,-0.238095,0.125,-0.111111,0.000000,0.0,0.0,0.0,...,-1.00,0.200000,NaN,0.6,0.935484,NaN,-0.4,0.274194,unknown/other,1
9,200063.0,NaN,0.0000,1.619048,0.375,1.444444,1.000000,0.0,0.0,0.0,...,-1.75,-1.100000,0.545455,0.8,-0.274194,-1.272727,-2.0,-0.370968,unknown/other,1


time: 69.9 ms


In [46]:
##saving
pd.DataFrame(worst_df).to_csv(Path(
    wd+'/data/processed/merged/%s_worstdf_std.csv' %(date)),index=False)

time: 998 ms


# aggregation 2:
## 4-11-19: range, mean and stdev aggregations (newagg)

np.ptp = difference btween min and max


In [48]:
##choose here if wanna use standardization or raw values.
values="standardize"

time: 1.09 ms


In [49]:
big_range= big_noCat.loc[big_noCat.loc[:,'label'].isin(continuous),:]
table4 = pd.pivot_table(big_range, values=values, columns='label', index=['icustay_id'],aggfunc=[np.ptp, np.median, np.std], dropna=[False,False,False]) 
table4.columns=['_'.join(col).strip() for col in table4.columns.values]
table4=table4.reset_index()

time: 52.9 s


In [50]:
#grabing the rest of the variables not suitable for range or mean/std
remaining= big_noCat.loc[big_noCat.loc[:,'label'].isin(onetime+ordinal+sparse),:]
table5 = pd.pivot_table(remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 

time: 638 ms


In [51]:
#table5.columns
table5.columns = table5.columns.get_level_values(1)
table5=table5.reset_index()

time: 3.37 ms


In [52]:
table4.head()#.rename(columns={})

,icustay_id,ptp_bicarbonate,ptp_bilirubin,ptp_bun,ptp_calcium,ptp_chloride,ptp_creatinine,ptp_daily_sofa,ptp_diasbp,ptp_glucose,...,std_ph,std_platelet,std_potassium,std_ptt,std_resprate,std_sodium,std_spo2,std_sysbp,std_temperature,std_wbc
0,200012.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.277778,NaN,...,NaN,NaN,NaN,NaN,0.503421,NaN,0.355157,0.129624,0.337811,NaN
1,200014.0,1.0,NaN,0.476190,1.000000,0.875,0.222222,0.000000,2.777778,0.250000,...,0.299544,0.085800,0.164957,0.076444,1.000079,0.529150,0.349257,0.862796,0.706647,0.335236
2,200019.0,0.0,NaN,0.000000,NaN,0.000,0.111111,0.333333,3.833333,0.803571,...,NaN,0.043182,0.202031,NaN,0.871901,0.000000,1.886965,1.038963,0.435482,0.079835
3,200030.0,0.4,0.0,0.333333,1.636364,0.125,0.222222,1.666667,2.611111,3.571429,...,0.189758,0.076463,0.884485,0.131076,0.542263,0.251661,0.326499,0.442354,0.599855,0.116308
4,200033.0,1.2,NaN,0.285714,0.272727,1.000,0.333333,0.333333,2.388889,5.250000,...,0.129630,0.110884,0.321753,0.089977,0.754394,0.783156,0.279803,0.444738,0.730230,0.123188


time: 56.9 ms


In [53]:
## merging aggregations together
#first left join all different newagg together. 
table4=pd.merge(table4, table5, how='left')

time: 37.8 ms


In [54]:
list(table4)[1:20]

['ptp_bicarbonate',
 'ptp_bilirubin',
 'ptp_bun',
 'ptp_calcium',
 'ptp_chloride',
 'ptp_creatinine',
 'ptp_daily_sofa',
 'ptp_diasbp',
 'ptp_glucose',
 'ptp_heartrate',
 'ptp_hemoglobin',
 'ptp_inr',
 'ptp_lactate',
 'ptp_meanartpress',
 'ptp_mingcs',
 'ptp_pao2fio2Ratio',
 'ptp_ph',
 'ptp_platelet',
 'ptp_potassium']

time: 3.8 ms


In [55]:
### formatting categorical to wide format to match the aggregated numerical
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy()
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per patient per time. 

time: 275 ms


### merging categorical with aggregated

In [56]:
#using fxn written for worst_df to make newagg_df
newagg_df=merge_cat_agg(table4, big_categorical, twoclass=False) #using range/mean/stdev aggregates 


time: 66.2 ms


In [57]:
newagg_df.head()

,icustay_id,ptp_bicarbonate,ptp_bilirubin,ptp_bun,ptp_calcium,ptp_chloride,ptp_creatinine,ptp_daily_sofa,ptp_diasbp,ptp_glucose,...,leukocyte,nitrite,norepinephrine,o2_flow,phenylephrine,rrt,vasopressin,vent_recieved,ethnicity,gender
0,200012.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.277778,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,asian,0
1,200014.0,1.0,NaN,0.476190,1.000000,0.875,0.222222,0.000000,2.777778,0.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,unknown/other,1
2,200019.0,0.0,NaN,0.000000,NaN,0.000,0.111111,0.333333,3.833333,0.803571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,white/nonhispanic,0
3,200030.0,0.4,0.0,0.333333,1.636364,0.125,0.222222,1.666667,2.611111,3.571429,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,black,1
4,200033.0,1.2,NaN,0.285714,0.272727,1.000,0.333333,0.333333,2.388889,5.250000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,white/nonhispanic,1


time: 56.1 ms


In [58]:
list(newagg_df)

['icustay_id',
 'ptp_bicarbonate',
 'ptp_bilirubin',
 'ptp_bun',
 'ptp_calcium',
 'ptp_chloride',
 'ptp_creatinine',
 'ptp_daily_sofa',
 'ptp_diasbp',
 'ptp_glucose',
 'ptp_heartrate',
 'ptp_hemoglobin',
 'ptp_inr',
 'ptp_lactate',
 'ptp_meanartpress',
 'ptp_mingcs',
 'ptp_pao2fio2Ratio',
 'ptp_ph',
 'ptp_platelet',
 'ptp_potassium',
 'ptp_ptt',
 'ptp_resprate',
 'ptp_sodium',
 'ptp_spo2',
 'ptp_sysbp',
 'ptp_temperature',
 'ptp_wbc',
 'median_bicarbonate',
 'median_bilirubin',
 'median_bun',
 'median_calcium',
 'median_chloride',
 'median_creatinine',
 'median_daily_sofa',
 'median_diasbp',
 'median_glucose',
 'median_heartrate',
 'median_hemoglobin',
 'median_inr',
 'median_lactate',
 'median_meanartpress',
 'median_mingcs',
 'median_pao2fio2Ratio',
 'median_ph',
 'median_platelet',
 'median_potassium',
 'median_ptt',
 'median_resprate',
 'median_sodium',
 'median_spo2',
 'median_sysbp',
 'median_temperature',
 'median_wbc',
 'std_bicarbonate',
 'std_bilirubin',
 'std_bun',
 'std_cal

time: 4.71 ms


In [59]:
pd.DataFrame(newagg_df).to_csv(Path(
    wd+'/data/processed/merged/%s_newaggdf_std.csv' %(date)),index=False)

time: 3.17 s


In [60]:
newagg_df['leukocyte']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       1.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       1.0
25       0.0
26       1.0
27       0.0
28       0.0
29       0.0
        ... 
14448    0.0
14449    0.0
14450    1.0
14451    0.0
14452    0.0
14453    0.0
14454    0.0
14455    0.0
14456    0.0
14457    0.0
14458    0.0
14459    0.0
14460    0.0
14461    0.0
14462    0.0
14463    0.0
14464    0.0
14465    0.0
14466    0.0
14467    0.0
14468    1.0
14469    0.0
14470    0.0
14471    0.0
14472    0.0
14473    0.0
14474    0.0
14475    0.0
14476    0.0
14477    0.0
Name: leukocyte, Length: 14478, dtype: float64

time: 6.3 ms


In [61]:
wd+'/data/processed/merged/%s_newaggdf_std.csv'

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/merged/%s_newaggdf_std.csv'

time: 4.13 ms


# aggregation 3:
# grabbing min/max/median/sd for only values found to be important from combining variable importance for random forest, boosted trees, and logistic regression on the baseline clinical max/min data.


In [194]:
# important_variables=["temperature","sysbp","daily_sofa","meanartpress","bun","platelet","ptt","heartrate","pao2fio2Ratio","creatinine","first_admit_age"]
important_continuous=["temperature","sysbp","daily_sofa","meanartpress","bun","platelet","ptt","heartrate","creatinine"]
important_ordinal=["any_vasoactive", 'vent_recieved', "leukocyte","pao2fio2Ratio"]
important_onetime=['first_admit_age','weight']



time: 2.73 ms


In [195]:
# #continuous and ordinal variables
# big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous+ordinal),:].copy()
# big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 

time: 580 µs


In [196]:
big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()

,icustay_id,value
0,200012.0,0.0
1,200014.0,0.0
2,200019.0,0.0
3,200030.0,1.0
4,200033.0,1.0
5,200036.0,0.0
6,200055.0,0.0
7,200059.0,1.0
8,200061.0,0.0
9,200063.0,0.0


time: 1.62 s


In [197]:
##combining vasoactives
vaso_active_df=big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()
vaso_active_df['uom']="y/n"
vaso_active_df['label']="any_vasoactive"
vaso_active_df['delta']=pd.to_timedelta("0days")
vaso_active_df['source']="all_vasoactives"

vaso_active_df=pd.merge(vaso_active_df, final_pt_df2[["icustay_id",'subject_id',"t_0"]], how="left", left_on="icustay_id", right_on="icustay_id")
vaso_active_df=pd.merge(vaso_active_df, median_label, how="left").fillna(0)
vaso_active_df['standardize']=vaso_active_df['value']
vaso_active_df['raw_value']=vaso_active_df['value']
vaso_active_df.head()

,icustay_id,value,uom,label,delta,source,subject_id,t_0,median,iqr,standardize,raw_value
0,200012.0,0.0,y/n,any_vasoactive,0 days,all_vasoactives,28448,2153-12-23,0.0,0.0,0.0,0.0
1,200014.0,0.0,y/n,any_vasoactive,0 days,all_vasoactives,9514,2105-02-17,0.0,0.0,0.0,0.0
2,200019.0,0.0,y/n,any_vasoactive,0 days,all_vasoactives,21789,2178-07-09,0.0,0.0,0.0,0.0
3,200030.0,1.0,y/n,any_vasoactive,0 days,all_vasoactives,14987,2150-11-14,0.0,0.0,1.0,1.0
4,200033.0,1.0,y/n,any_vasoactive,0 days,all_vasoactives,56369,2198-08-11,0.0,0.0,1.0,1.0


time: 1.56 s


In [198]:
vaso_active_df.iloc[3,1]

'1.0'

time: 2.47 ms


In [199]:
big_noCat.head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id,median,iqr,raw_value,standardize
0,200012.0,0.693147,daily_sofa_score,daily_sofa,0 days 00:00:00.000000000,2153-12-23,sofa,28448,1.38629,1.38629,1,-0.500000
1,200012.0,0.832909,mmol/L,lactate,0 days 02:36:00.000000000,2153-12-23,labs,28448,0.993252,0.875469,1.3,-0.183151
2,200012.0,2.77259,gcs_score,mingcs,0 days 05:12:00.000000000,2153-12-23,gcs,28448,2.70805,1.09861,15,0.058745
3,200012.0,4.38203,mmHg,diasbp,0 days 06:16:00.000000000,2153-12-23,vitals,28448,4.09434,2.94444,79,0.097704
4,200012.0,4.70048,bpm,heartrate,0 days 06:16:00.000000000,2153-12-23,vitals,28448,4.45435,3.17805,109,0.077448


time: 19.4 ms


In [200]:
big_noCat['label'].unique()
#big_noCat=pd.concat([big_noCat, vaso_active_df], sort=False)

array(['daily_sofa', 'lactate', 'mingcs', 'diasbp', 'heartrate',
       'meanartpress', 'resprate', 'spo2', 'sysbp', 'temperature',
       'bands', 'hemoglobin', 'platelet', 'wbc', 'calcium', 'glucose',
       'pco2', 'ph', 'bicarbonate', 'bun', 'chloride', 'creatinine',
       'inr', 'potassium', 'ptt', 'sodium', 'bilirubin', 'dobutamine',
       'dopamine', 'epinephrine', 'norepinephrine', 'o2_flow',
       'pao2fio2Ratio', 'phenylephrine', 'rrt', 'vasopressin',
       'vent_recieved', 'leukocyte', 'nitrite', 'any_vasoactive'],
      dtype=object)

time: 301 ms


In [201]:
big_noCat[big_noCat['label']=="any_vasoactive"].head() #so this works

,icustay_id,value,uom,label,delta,t_0,source,subject_id,median,iqr,raw_value,standardize
0,200012.0,0.0,y/n,any_vasoactive,0 days 00:00:00,2153-12-23,all_vasoactives,28448,0,0,0.0,0.0
1,200014.0,0.0,y/n,any_vasoactive,0 days 00:00:00,2105-02-17,all_vasoactives,9514,0,0,0.0,0.0
2,200019.0,0.0,y/n,any_vasoactive,0 days 00:00:00,2178-07-09,all_vasoactives,21789,0,0,0.0,0.0
3,200030.0,1.0,y/n,any_vasoactive,0 days 00:00:00,2150-11-14,all_vasoactives,14987,0,0,1.0,1.0
4,200033.0,1.0,y/n,any_vasoactive,0 days 00:00:00,2198-08-11,all_vasoactives,56369,0,0,1.0,1.0


time: 446 ms


In [202]:
##choose here if wanna use standardization or raw values.
values="standardize"
#big_noCat[values]= big_noCat[values].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better.

time: 932 µs


In [203]:
# big_df2=pd.concat([big_df, vaso_active_df], sort=False)

# #splitting categorical, ordinal and continuous
# big_categorical= big_df.loc[big_df.loc[:,'label'].isin(important_ordinal+important_onetime),:].copy()

# #continuous and ordinal variables
# df_continuous= big_df.loc[big_df.loc[:,'label'].isin(important_continuous+ordinal),:].copy()
# big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 

time: 3.36 ms


In [204]:
df_continuous= big_noCat.loc[big_noCat.loc[:,'label'].isin(important_continuous),:]
agg_table1 = pd.pivot_table(df_continuous, values=values, columns='label', index=['icustay_id'],aggfunc=[np.max, np.min, np.median, np.std], dropna=[False,False,False, False]) 
agg_table1.columns=['_'.join(col).strip() for col in agg_table1.columns.values]
agg_table1=agg_table1.reset_index()

time: 2.64 s


In [205]:
#code above is not 

time: 772 µs


In [206]:
#grabing the rest of the variables not suitable for range or mean/std
big_df2=pd.concat([big_df, vaso_active_df], sort=False)

agg_remaining= big_df2.loc[big_df2.loc[:,'label'].isin(important_onetime+important_ordinal),:]
agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
agg_table2.columns = agg_table2.columns.get_level_values(1)
agg_table2=agg_table2.reset_index()
agg_table2.head()#.rename(columns={})

label,icustay_id,any_vasoactive,first_admit_age,leukocyte,pao2fio2Ratio,vent_recieved,weight
0,200012.0,0.0,32.99,0,"(475, 3000]",0,51.2
1,200014.0,0.0,84.73,0,"(200, 333]",1,62.0
2,200019.0,0.0,82.88,0,"(333, 475]",1,65.0
3,200030.0,1.0,54.19,0,"(333, 475]",0,113.6
4,200033.0,1.0,67.14,0,"(475, 3000]",1,74.0


time: 25.8 s


In [207]:
## merging aggregations together
#first left join all different newagg together. 
agg_cont_cat=pd.merge(agg_table1, agg_table2, how='left')

time: 20.2 ms


In [208]:
list(agg_cont_cat)

['icustay_id',
 'amax_bun',
 'amax_creatinine',
 'amax_daily_sofa',
 'amax_heartrate',
 'amax_meanartpress',
 'amax_platelet',
 'amax_ptt',
 'amax_sysbp',
 'amax_temperature',
 'amin_bun',
 'amin_creatinine',
 'amin_daily_sofa',
 'amin_heartrate',
 'amin_meanartpress',
 'amin_platelet',
 'amin_ptt',
 'amin_sysbp',
 'amin_temperature',
 'median_bun',
 'median_creatinine',
 'median_daily_sofa',
 'median_heartrate',
 'median_meanartpress',
 'median_platelet',
 'median_ptt',
 'median_sysbp',
 'median_temperature',
 'std_bun',
 'std_creatinine',
 'std_daily_sofa',
 'std_heartrate',
 'std_meanartpress',
 'std_platelet',
 'std_ptt',
 'std_sysbp',
 'std_temperature',
 'any_vasoactive',
 'first_admit_age',
 'leukocyte',
 'pao2fio2Ratio',
 'vent_recieved',
 'weight']

time: 1.79 ms


In [209]:
agg_cont_cat['pao2fio2Ratio'].value_counts() 

(475, 3000]    9903
(0, 200]       1885
(200, 333]     1422
(333, 475]     1260
Name: pao2fio2Ratio, dtype: int64

time: 7.83 ms


In [210]:
agg_cont_cat['any_vasoactive'].value_counts() 

0.0    9880
1.0    4590
Name: any_vasoactive, dtype: int64

time: 5.69 ms


In [211]:
agg_cont_cat.head()

,icustay_id,amax_bun,amax_creatinine,amax_daily_sofa,amax_heartrate,amax_meanartpress,amax_platelet,amax_ptt,amax_sysbp,amax_temperature,...,std_platelet,std_ptt,std_sysbp,std_temperature,any_vasoactive,first_admit_age,leukocyte,pao2fio2Ratio,vent_recieved,weight
0,200012.0,NaN,NaN,-0.500000,0.077448,0.047571,-0.076639,NaN,-0.022901,0.021964,...,NaN,NaN,0.011027,0.012918,0.0,32.99,0,"(475, 3000]",0,51.2
1,200014.0,0.056406,-0.164150,-0.207519,0.021221,0.263979,-0.067398,-0.030164,0.118889,0.003685,...,0.018384,0.013532,0.061265,0.028030,0.0,84.73,0,"(200, 333]",1,62.0
2,200019.0,0.056406,0.000000,0.403677,-0.064773,0.150264,-0.054275,-0.020962,0.161211,0.032017,...,0.008219,NaN,0.069096,0.016760,0.0,82.88,0,"(333, 475]",1,65.0
3,200030.0,0.104422,0.000000,0.292481,0.132424,0.316786,-0.011133,-0.022099,0.125324,0.043589,...,0.012143,0.023367,0.027687,0.023041,1.0,54.19,0,"(333, 475]",0,113.6
4,200033.0,-0.068362,-0.253202,-0.500000,0.122666,0.125991,-0.061462,-0.034854,0.084386,0.061749,...,0.023777,0.016384,0.030143,0.027794,1.0,67.14,0,"(475, 3000]",1,74.0


time: 21.2 ms


In [212]:
pd.DataFrame(agg_cont_cat).to_csv(Path(
    wd+'/data/processed/merged/%s_newaggdf2_std.csv' %(date)),index=False)

time: 1.01 s


In [ ]:
#YUS, got this reprocessed using a few of the top variables  4-16-19

'04042019'

time: 2.33 ms


In [ ]:
# big_categorical["icustay_id"].isin(icustay_id)

In [ ]:
#big_noCat['value']=big_noCat['value'].astype(float, copy=False)
#big_noCat.groupby(['icustay_id','label'], as_index=False)['value'].agg(dict1) #

In [ ]:
# test= big_noCat.groupby(['icustay_id'], as_index=False)['value'].agg(dict1)
# table = pd.pivot_table(big_noCat, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
# table

# pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
#                aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

In [ ]:
# #table = pivot_table(df, values=['D', 'E'], index=['A', 'C'],aggfunc={})
# ##don't know why this doesn't work. 
# table = pd.pivot_table(big_noCat, values='value',columns='label', index='icustay_id',aggfunc=dict1, dropna=False)
# table


In [ ]:
# big_agg= big_df.loc[big_df.loc[:,'label'].isin(non_category),:].groupby(['icustay_id','label'], as_index=False)['value'].apply(lambda x: float(x))

    #.agg({str(x):['max'] for x in hi_value})
#big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

In [ ]:
# hi_df= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]
# hi_df['value']=hi_df['value'].astype(float, copy=False)
# hi_df.groupby(['icustay_id','label'], as_index=False)['value'].agg('max') #this works

In [ ]:
# %whos

# del big_agg
# del hi_df
# del final_pt_df2